# TODO

you can't actually test this model therefore create a version with just df

In [1]:
import os
import sklearn
import seaborn as sns
import numpy as np
import pandas as pd
import re
import pickle
import json
import math
import random
from tqdm import tqdm
import datetime
import time

from PyAstronomy import pyasl

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, MissingIndicator

from sklearn.preprocessing import (
    StandardScaler, 
    MinMaxScaler, 
    MaxAbsScaler, 
    # RobustScalar,
    Normalizer,
    QuantileTransformer,
    PowerTransformer,
    OneHotEncoder, 
    OrdinalEncoder,
    LabelEncoder
)

from sklearn.utils import all_estimators

from sklearn.base import (
    RegressorMixin, 
    ClassifierMixin,
    TransformerMixin
)

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    auc,
    roc_auc_score,
    f1_score,
    r2_score,
    mean_squared_error,
    classification_report
)

import warnings
import xgboost
import catboost
import lightgbm

import tensorflow as tf

warnings.filterwarnings("ignore")
pd.set_option("display.precision", 2)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

removed_classifiers = [
    "ClassifierChain",
    "ComplementNB",
    # "GradientBoostingClassifier",
    "GaussianProcessClassifier",
    "HistGradientBoostingClassifier",
    # "MLPClassifier",
    "LogisticRegressionCV", 
    "MultiOutputClassifier", 
    "MultinomialNB", 
    "OneVsOneClassifier",
    "OneVsRestClassifier",
    "OutputCodeClassifier",
    "RadiusNeighborsClassifier",
    "VotingClassifier",
    "CategoricalNB",
    "StackingClassifier",
    "NuSVC",
]

removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

CLASSIFIERS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], ClassifierMixin) and (est[0] not in removed_classifiers))
]


REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]

REGRESSORS.append(("XGBRegressor", xgboost.XGBRegressor))
REGRESSORS.append(("LGBMRegressor", lightgbm.LGBMRegressor))
REGRESSORS.append(('CatBoostRegressor', catboost.CatBoostRegressor))

CLASSIFIERS.append(("XGBClassifier", xgboost.XGBClassifier))
CLASSIFIERS.append(("LGBMClassifier", lightgbm.LGBMClassifier))
CLASSIFIERS.append(('CatBoostClassifier', catboost.CatBoostClassifier))

TRANSFOMER_METHODS = [
    ("StandardScaler", StandardScaler), 
    ("MinMaxScaler", MinMaxScaler), 
    ("MaxAbsScaler", MaxAbsScaler), 
    # ("RobustScalar", RobustScalar),
    ("Normalizer", Normalizer),
    ("QuantileTransformer", QuantileTransformer),
    ("PowerTransformer", PowerTransformer),
]

def adjusted_rsquared(r2, n, p):
    return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

2023-05-21 22:51:40.788576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:51:40.942583: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-21 22:51:41.390697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-21 22:51:41.390788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

References:

1. https://discuss.huggingface.co/t/what-does-increasing-number-of-heads-do-in-the-multi-head-attention/1847/2

In [2]:
df_train = pd.read_excel("dataset.xlsx", sheet_name="train")
df_test = pd.read_excel("dataset.xlsx", sheet_name="test")
display(df_train)
display(df_test)

,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max
0,Al92Nd6Ni2,10.61,0.32,0.20,-0.17,0.00
1,Zr26Ti10Cu8Ni8Be20Y4Mg24,13.89,1.78,0.17,-0.12,5.00
2,Gd55Ni22Mn3Al20,11.82,1.09,0.26,-0.18,2.00
3,Ag53.8Mg15.4Ca30.8,10.05,0.98,0.25,-0.28,0.80
4,Ni60Nb20Zr20,19.88,0.95,0.17,-0.13,0.50
...,...,...,...,...,...,...
1054,Mg65Cu7.5Ni7.5Ag5Zn5Gd7.5Y2.5,9.89,1.25,0.18,-0.16,13.00
1055,Mg65Cu7.5Ni7.5Ag5Zn5Gd2.5Y7.5,9.96,1.25,0.17,-0.16,9.50
1056,Mg80Ni10Nd10,9.39,0.64,0.25,-0.09,0.60
1057,Cu45.5Mg31.8Ca22.7,10.66,1.06,0.34,-0.25,1.25


,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max,predicted_d_max
0,Co14Ni69P17,14.24,0.83,0.12,0.04,0.00,0.02
1,C14Fe66W20,30.81,0.87,0.14,-0.13,0.00,0.00
2,Cu15La30Mg55,8.64,0.97,0.31,-0.13,0.00,1.77
3,Al87.5Ca2.5Ni10,11.30,0.44,0.21,-0.24,0.00,-0.01
4,B36C8Co56,35.47,0.89,0.11,-0.14,0.00,-0.03
...,...,...,...,...,...,...,...
257,Gd55Ni25Al20,11.94,1.00,0.27,-0.19,2.00,1.36
258,Mg65Ni20La15,10.04,0.89,0.29,-0.14,0.50,0.44
259,B8Fe89Ni3,16.80,0.41,0.16,-0.02,0.00,0.02
260,La62Al14Cu18Ag6,8.44,1.05,0.36,-0.23,5.00,5.77


In [3]:
df_test = df_test.drop(["predicted_d_max"], axis=1)
df = pd.concat([df_train, df_test], axis=0)
df = df.reset_index(drop=True)
display(df)

,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max
0,Al92Nd6Ni2,10.61,0.32,0.20,-0.17,0.00
1,Zr26Ti10Cu8Ni8Be20Y4Mg24,13.89,1.78,0.17,-0.12,5.00
2,Gd55Ni22Mn3Al20,11.82,1.09,0.26,-0.18,2.00
3,Ag53.8Mg15.4Ca30.8,10.05,0.98,0.25,-0.28,0.80
4,Ni60Nb20Zr20,19.88,0.95,0.17,-0.13,0.50
...,...,...,...,...,...,...
1316,Gd55Ni25Al20,11.94,1.00,0.27,-0.19,2.00
1317,Mg65Ni20La15,10.04,0.89,0.29,-0.14,0.50
1318,B8Fe89Ni3,16.80,0.41,0.16,-0.02,0.00
1319,La62Al14Cu18Ag6,8.44,1.05,0.36,-0.23,5.00


## 1. Test the capability of our d_max predictor

In [4]:
# constants

n_elements = 118
alloy_max_len = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

max_factor = max(df["actual_d_max"])

print(max_factor)

with open("parameters.json") as json_file:
    parameters = json.load(json_file)

30.0


In [5]:
# basic utility functions to calculate features

def get_elements_and_compositions(x, verbose = -1):
    # seperating atoms from composition
    s = re.sub(r'[^\w\s]','',x)
    s = re.sub('\d',' ',s)
    elements = np.array([i for i in s.split(' ') if i != ""])
    if verbose > 0:
        print('\nElements in BMG are : ', elements)

    compositions = re.findall(r"[-+]?\d*\.\d+|\d+", x)
    compositions = [float(i) for i in compositions]
    if verbose > 0:
        print('Compositions: ', compositions)
    
    return elements, compositions

def diff(alloy):
    # making ranges for each atom
    ranges = {}
    for i in alloy:
        ranges[i] = 0.88 * parameters[i]["ar"]
    # compiling scoring matrix
    score = {}
    for i in alloy:
        current_score = {}
        for j in alloy:
            if parameters[i]["ar"] < ranges[j]:
                current_score[j] = -1
            elif parameters[i]["ar"] > parameters[j]["ar"]:
                current_score[j] = 1
            else:
                current_score[j] = 0
        score[i] = current_score

    big = []
    small = []
    # separating into big and small based on scoring matrix
    for i in score:
        total_sum = 0
        for j in score[i]:
            total_sum = total_sum + score[i][j]
        if total_sum > 0:
            big.append(i)
        else:
            small.append(i)

    if len(big) == 0 or len(small) == 0:
        print(score)
    return big, small
    
# finds the paramater deltaE
def electro(elements_and_compositions, alloy):
    summation_of_product_of_composition_and_electronegativity = 0 # summation of product of composition and electro negativity
    summation_of_composition = 0 # summation of compositions
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_composition = summation_of_composition + elements_and_compositions[i]
            
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_product_of_composition_and_electronegativity = summation_of_product_of_composition_and_electronegativity + elements_and_compositions[i] * parameters[i]['en']
    return summation_of_product_of_composition_and_electronegativity / summation_of_composition

# finds the paramater deltaD
def comps(elements_and_compositions, alloy):
    summation_of_product_of_composition_and_atomicradii = 0 # summation of product of composition and atomic radii
    summation_of_compositions = 0 # summation of compositions
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_compositions = summation_of_compositions + elements_and_compositions[i]
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_product_of_composition_and_atomicradii = summation_of_product_of_composition_and_atomicradii + elements_and_compositions[i] * parameters[i]['ar']
    return summation_of_product_of_composition_and_atomicradii / summation_of_compositions

def prepare_params(alloy):
    # seperating atoms from composition
    s = re.sub(r'[^\w\s]','', alloy)
    s = re.sub('\d', ' ', s)
    elements = np.array([i for i in s.split(' ') if i in parameters]) # elements list
    # print('\nElements in BMG are : ', elements)

    compositions = re.findall(r"[-+]?\d*\.\d+|\d+", alloy)
    compositions = [float(i) for i in compositions]
    # print('Compositions: ', compositions)

    elements_and_compositions = dict(zip(elements, compositions))
    s_mix = 0
    h_mix = 0

    for i in elements_and_compositions:
        s_mix = s_mix + (elements_and_compositions[i] / 100) * (math.log((elements_and_compositions[i] / 100)))
        h_mix = h_mix + (elements_and_compositions[i] / 100) * parameters[i]['enthalphy']
    s_mix = -1*s_mix

    big, small = diff(elements)
    # print("big atoms : ", big)
    # print("small atoms : ", small)
    delta_d = (comps(elements_and_compositions,big) - comps(elements_and_compositions,small)) / (comps(elements_and_compositions,big))
    delta_e = (electro(elements_and_compositions,big) - electro(elements_and_compositions,small)) / (electro(elements_and_compositions,big) + electro(elements_and_compositions,small))
    return h_mix, s_mix, delta_d, delta_e, elements[np.argmax(compositions)]

In [37]:
# utility functions for encoding

# find element index from all_elements, e.g. "He" = 2
def element_to_index(element):
    try:
        atomic_number = pyasl.AtomicNo()
        return atomic_number.getAtomicNo(element)
    except:
        return "END"
    
def index_to_element(index):
    try:
        atomic_number = pyasl.AtomicNo()
        return atomic_number.getElSymbol(index)
    except:
        return "END"
    
def alloy_to_1d_tensor(alloy_str, alloy_max_len = alloy_max_len):
    tensor = torch.zeros(alloy_max_len)
    elements, compositions = get_elements_and_compositions(alloy_str)
    i = 0
    for idx in range(0, len(elements) + len(compositions), 2):
        tensor[idx] = element_to_index(elements[i])
        tensor[idx + 1] = compositions[i]
        i += 1
    return tensor

def elements_to_1d_tensor(elements, element_max_len = alloy_max_len // 2):
    tensor = torch.zeros(element_max_len)
    for idx in range(len(elements)):
        tensor[idx] = element_to_index(elements[idx])
    # print(tensor)
    return tensor

def composition_to_1d_tensor(compositions, composition_max_len = alloy_max_len // 2):
    tensor = torch.zeros(composition_max_len)
    for idx in range(len(compositions)):
        tensor[idx] = compositions[idx]
    # print(tensor)
    return tensor

def combine_elements_and_composition_to_alloy(elements, compositions):
    alloy = ""
    for idx in range(len(elements)):
        alloy += str(index_to_element(elements[idx])) + str(compositions[idx])[:4]
    return alloy.split("END")[0]

def combine_elements_and_composition_to_alloy_already_string(elements, compositions):
    alloy = ""
    for idx in range(len(elements)):
        alloy += str(elements[idx]) + str(compositions[idx])[:4]
    return alloy.split("END")[0]

In [7]:
class SequenceToSigmoid(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

def predict_d_max(alloy):
    input_size = 20
    hidden_size = 128
    learning_rate = 0.001
    num_epochs = 2000
    batch_size = 32
    num_layers = 3
    
    loaded_rnn_encoder = SequenceToSigmoid(input_size, hidden_size, num_layers)
    loaded_rnn_encoder.load_state_dict(torch.load("rnn_fully_trained.pt"))
    loaded_rnn_encoder.eval()
    
    loaded_X_transformer = pickle.load(open("catboost_X_quantiletransformer.pkl", "rb"))
    loaded_y_transformer = pickle.load(open("catboost_y_quantiletransformer.pkl", "rb"))
    loaded_catboost_predictor = pickle.load(open("catboost_model.pkl", "rb"))
    
    alloy_tensor = alloy_to_1d_tensor(alloy)
    rnn_prediction = loaded_rnn_encoder(torch.atleast_2d(alloy_tensor)).squeeze().item()
    rnn_prediction *= max_factor
    # print(rnn_prediction)
    
    h_mix, s_mix, delta_d, delta_e, max_element = prepare_params(alloy)
    # print(h_mix, s_mix, delta_d, delta_e, max_element)
    
    transformed_input = loaded_X_transformer.transform(np.array([h_mix, s_mix, delta_d, delta_e, rnn_prediction]).reshape(1, -1))
    # print(transformed_input)
    transformed_output = loaded_catboost_predictor.predict(transformed_input)
    # print(transformed_output)
    output = loaded_y_transformer.inverse_transform(transformed_output.reshape(1, -1)).item()
    return output

In [8]:
predict_d_max("Zr26Ti10Cu8Ni8Be20Y4Mg24")

5.0

## 2. Generate optimized BMGs

In [9]:
X_train = []
y_train = []

X_test = []
y_test = []

for i in range(df.shape[0]):
    elements, compositions = get_elements_and_compositions(df.loc[i, "bmg_alloy"])
    X_train.append(elements_to_1d_tensor(elements))
    y_train.append(composition_to_1d_tensor(compositions))

X_train = torch.stack(X_train)
y_train = torch.stack(y_train)

print(X_train.shape, X_train.dtype, y_train.shape, y_train.dtype)

torch.Size([1321, 10]) torch.float32 torch.Size([1321, 10]) torch.float32


In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()

        self.hidden_size = hidden_size

        # Encoder
        self.encoder = nn.LSTM(input_size, hidden_size)

        # Decoder
        self.decoder = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, target_seq):
        
        input_seq = input_seq.unsqueeze(1)
        target_seq = target_seq.unsqueeze(1)
        
        # Encode the input sequence
        _, hidden = self.encoder(input_seq)

        # Decode the output sequence
        output, _ = self.decoder(target_seq, hidden)
        output = self.fc(output)
        
        output = F.softmax(output, dim=2)
        output = output * (input_seq != 0).float()

        # Normalize the output sequence to sum up to 1
        output_sum = output.sum(dim=2, keepdim=True)
        output = output / output_sum
        
        return output.squeeze(1)

def custom_loss(output, target, generated_alloy_d_max, actual_alloy_d_max):
    # TODO: define the strategy of the loss here
    normalized_d_max = generated_alloy_d_max / max_factor
    return Variable(torch.tensor(normalized_d_max * -1), requires_grad = True)

In [11]:
X_train[1192].view(1, -1), y_train[1192].view(1, -1)

(tensor([[12., 28., 57.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 tensor([[60.0000, 23.6000, 16.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]]))

In [12]:
input_size = alloy_max_len // 2
hidden_size = 32
output_size = alloy_max_len // 2

model = Seq2Seq(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def random_training_example(input_data, output_data):
    # print(input_data.shape, output_data.shape)
    random_number = random.randint(0, input_data.shape[0] - 1)
    # print(random_number)
    elements_tensor, compositions_tensor = input_data[random_number].view(1, -1), output_data[random_number].view(1, -1)
    
    # print(elements_tensor, compositions_tensor)
    element_input = elements_tensor[0].tolist()
    composition_output = compositions_tensor[0].tolist()
    
    elements = []
    for idx in range(len(element_input)):
        elements.append(index_to_element(element_input[idx]))
    alloy_str = ""
    
    for idx in range(min(len(composition_output), len(elements))):
        alloy_str += str(elements[idx]) + str(composition_output[idx])
    
    alloy_str = alloy_str.split("END")[0]
    # print("Actual alloy:", alloy_str)
    # print("Actual d_max:", df_train.loc[random_number, "actual_d_max"])
    
    return elements_tensor, compositions_tensor, alloy_str, df.loc[random_number, "actual_d_max"]

random_training_example(X_train, y_train)

(tensor([[27., 26.,  5., 14., 41.,  0.,  0.,  0.,  0.,  0.]]),
 tensor([[57.6000, 14.4000, 19.2000,  4.8000,  4.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]]),
 'Co57.599998474121094Fe14.399999618530273B19.200000762939453Si4.800000190734863Nb4.0',
 2.5)

In [13]:
num_epochs = 250000
all_losses = []
current_loss = 0
plot_every = 100
print_every = 1000

wins_and_losses = []
better_generations = []
wins_tracker = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    rand_input_seq, rand_output_seq, actual_alloy, actual_alloy_d_max = random_training_example(X_train, y_train)
    output_seq = model(rand_input_seq, rand_input_seq)
    # print(np.sum(np.array(output_seq[0].tolist()) * 100))
    generated_alloy = combine_elements_and_composition_to_alloy(rand_input_seq[0].tolist(), list(np.array(output_seq[0].tolist()) * 100))
    # print("Generated Alloy", generated_alloy)
    generated_alloy_d_max = predict_d_max(generated_alloy)
    # print("Generated Alloy d_max", generated_alloy_d_max)
    # print("LOSS", loss)

    if generated_alloy_d_max > actual_alloy_d_max:
        wins_and_losses.append("win")
        better_generations.append(generated_alloy)
        wins_tracker.append({"actual_alloy": actual_alloy,
                             "actual_alloy_d_max": actual_alloy_d_max,
                             "generated_alloy": generated_alloy,
                             "generated_alloy_d_max": generated_alloy_d_max,
                             "gain": generated_alloy_d_max - actual_alloy_d_max
                            })
    else:
        wins_and_losses.append("loss")

    loss = custom_loss(output_seq, rand_output_seq, generated_alloy_d_max, actual_alloy_d_max)
    # loss = Variable(0.0, requires_grad = True) if generated_alloy_d_max > actual_d_max else Variable(1.0, requires_grad = True)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch + 1) % print_every == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
    if (epoch + 1) % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

Epoch [1000/250000], Loss: -0.0
Epoch [2000/250000], Loss: -0.0
Epoch [3000/250000], Loss: -0.0
Epoch [4000/250000], Loss: -0.13333334028720856
Epoch [5000/250000], Loss: -0.0
Epoch [6000/250000], Loss: -0.0
Epoch [7000/250000], Loss: -0.0
Epoch [8000/250000], Loss: -0.06666667014360428
Epoch [9000/250000], Loss: -0.1666666716337204
Epoch [10000/250000], Loss: -0.0
Epoch [11000/250000], Loss: -0.0
Epoch [12000/250000], Loss: -0.13333334028720856
Epoch [13000/250000], Loss: -0.10000000149011612
Epoch [14000/250000], Loss: -0.0
Epoch [15000/250000], Loss: -0.10000000149011612
Epoch [16000/250000], Loss: -0.0
Epoch [17000/250000], Loss: -0.0
Epoch [18000/250000], Loss: -0.0
Epoch [19000/250000], Loss: -0.13333334028720856
Epoch [20000/250000], Loss: -0.10000000149011612
Epoch [21000/250000], Loss: -0.0
Epoch [22000/250000], Loss: -0.0
Epoch [23000/250000], Loss: -0.0
Epoch [24000/250000], Loss: -0.10000000149011612
Epoch [25000/250000], Loss: -0.0
Epoch [26000/250000], Loss: -0.0
Epoch [2

Epoch [201000/250000], Loss: -0.10000000149011612
Epoch [202000/250000], Loss: -0.1666666716337204
Epoch [203000/250000], Loss: -0.06666667014360428
Epoch [204000/250000], Loss: -0.0
Epoch [205000/250000], Loss: -0.0
Epoch [206000/250000], Loss: -0.2666666805744171
Epoch [207000/250000], Loss: -0.1666666716337204
Epoch [208000/250000], Loss: -0.01666666753590107
Epoch [209000/250000], Loss: -0.0
Epoch [210000/250000], Loss: -0.0
Epoch [211000/250000], Loss: -0.10000000149011612
Epoch [212000/250000], Loss: -0.0
Epoch [213000/250000], Loss: -0.0
Epoch [214000/250000], Loss: -0.0
Epoch [215000/250000], Loss: -0.10000000149011612
Epoch [216000/250000], Loss: -0.0
Epoch [217000/250000], Loss: -0.1635189950466156
Epoch [218000/250000], Loss: -0.0
Epoch [219000/250000], Loss: -0.10000000149011612
Epoch [220000/250000], Loss: -0.06666667014360428
Epoch [221000/250000], Loss: -0.0
Epoch [222000/250000], Loss: -0.0
Epoch [223000/250000], Loss: -0.0
Epoch [224000/250000], Loss: -0.33333334326744

In [14]:
from collections import Counter
Counter(wins_and_losses)

Counter({'loss': 195066, 'win': 54934})

In [15]:
print(better_generations)

['Fe16.3Nb31.2B14.6Si20.5Y17.1', 'Ni14.8Zr36.5Ti14.6Nb21.4Al12.4', 'Ce24.3Cu49.6Mg26.0', 'Al31.0Ce48.1Ni20.7', 'Al29.6Gd49.6Ni20.6', 'Ag15.0Mg34.6Ca17.8Cu32.4', 'Al31.3La47.8Ni20.7', 'Al30.3Ce49.0Fe20.6', 'Ce24.3Cu49.6Mg26.0', 'Ti18.3Zr26.3Hf18.9Cu19.8Ni16.4', 'Fe15.1Mo23.1Nb11.6P19.0C12.0B19.0', 'Al35.1Ca64.8', 'Ni19.4Zr19.2Ti15.3Si13.4Sn12.2Nb20.1', 'Cu29.2Hf50.8Ti19.8', 'Al34.4Ce41.7Nb23.8', 'Ni19.4Zr19.2Ti15.3Si13.4Sn12.2Nb20.1', 'Ti20.7Cu36.1Zr17.0Ni26.1', 'Al29.6Gd49.6Ni20.6', 'Al30.0Ce49.3Mn20.6', 'Ni19.4Zr19.2Ti15.3Si13.4Sn12.2Nb20.1', 'Y22.1Fe36.0B22.2Ta19.4', 'Ti15.8Cu17.3Ni13.7Al12.5Si11.8Sc16.2B12.4', 'Fe34.9Zr65.0', 'Fe13.1Mo25.4P10.0C22.0B13.1Si16.1', 'Tm15.5Y33.6Al16.9Co33.9', 'Fe11.6Co17.9Cr11.0Mo15.7C11.2B18.5Er13.8', 'Al31.4Ce47.7Cu20.8', 'Co16.9Fe27.1B15.5Si22.4Nb17.9', 'Zr14.6Nb30.0Cu13.1Ni28.1Be13.9', 'Zr16.7Al29.5Co14.5Cu25.0Ag13.9', 'Fe17.9Cr25.8Mo16.3C23.3B16.3', 'Fe20.7B35.7Si17.7Nb25.7', 'Ni19.4Zr19.2Ti15.3Si13.4Sn12.2Nb20.1', 'Al31.3La47.8Ni20.7', 'Fe23.8Mo34

In [16]:
for i in wins_tracker:
    print(i)
    print("\n")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'actual_alloy': 'Ni60.0Zr20.0Ti7.5Nb7.5Al5.0', 'actual_alloy_d_max': 2.0, 'generated_alloy': 'Ni14.8Zr36.5Ti14.6Nb21.4Al12.4', 'generated_alloy_d_max': 4.0, 'gain': 2.0}


{'actual_alloy': 'Cu60.0Hf17.5Ti22.5', 'actual_alloy_d_max': 3.0, 'generated_alloy': 'Cu29.2Hf50.8Ti19.8', 'generated_alloy_d_max': 4.929819443959293, 'gain': 1.9298194439592926}


{'actual_alloy': 'Ni42.0Ti20.0Zr25.0Al8.0Cu5.0', 'actual_alloy_d_max': 0.5, 'generated_alloy': 'Ni18.6Ti22.6Zr19.2Al21.3Cu18.0', 'generated_alloy_d_max': 5.0, 'gain': 4.5}


{'actual_alloy': 'Fe52.650001525878906Co5.849999904632568Cr6.0Mo14.0C19.0B2.0Er0.5', 'actual_alloy_d_max': 2.0, 'generated_alloy': 'Fe11.6Co17.9Cr11.0Mo15.7C11.2B18.5Er13.8', 'generated_alloy_d_max': 5.0, 'gain': 3.0}


{'actual_alloy': 'Ni42.0Ti19.0Zr22.5Al8.0Cu5.0Si3.5', 'actual_alloy_d_max': 3.0, 'generated_alloy': 'Ni13.5Ti20.7Zr17.3Al14.9Cu14.0Si19.4', 'generated_alloy_d_max': 5.0, 'gain': 2.0}


{'actual_alloy': 'Fe76.0Si9.0B10.0P5.0', 'actual_alloy_d_max': 2.5,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'actual_alloy': 'Ag46.20000076293945Mg15.399999618530273Ca30.700000762939453Cu7.699999809265137', 'actual_alloy_d_max': 0.8, 'generated_alloy': 'Ag15.0Mg34.6Ca17.8Cu32.4', 'generated_alloy_d_max': 3.0, 'gain': 2.2}


{'actual_alloy': 'Fe72.0C7.0Si3.299999952316284B5.0P8.699999809265137Ga4.0', 'actual_alloy_d_max': 2.0, 'generated_alloy': 'Fe15.2C18.3Si15.8B15.1P15.1Ga20.2', 'generated_alloy_d_max': 4.0, 'gain': 2.0}


{'actual_alloy': 'Fe4.0Zr48.0Cu32.0Al8.0Ag8.0', 'actual_alloy_d_max': 5.0, 'generated_alloy': 'Fe16.4Zr25.3Cu18.6Al21.7Ag17.7', 'generated_alloy_d_max': 8.0, 'gain': 3.0}


{'actual_alloy': 'Fe70.0Ni2.0Y6.0B22.0', 'actual_alloy_d_max': 2.0, 'generated_alloy': 'Fe20.8Ni32.4Y17.9B28.7', 'generated_alloy_d_max': 2.983223526851166, 'gain': 0.983223526851166}


{'actual_alloy': 'Cu60.0Hf20.0Ti20.0', 'actual_alloy_d_max': 4.0, 'generated_alloy': 'Cu29.2Hf50.8Ti19.8', 'generated_alloy_d_max': 4.929819443959293, 'gain': 0.9298194439592926}


{'actual_alloy': 'Al14.0Ce8.0Cu78.0',

In [18]:
torch.save(model.state_dict(), "bmg_generator.pt")

In [46]:
window = 5
n = 3  # Number of nested loops
all_combinations = []

def generate_combinations(curr_combination, remaining_loops):
    if remaining_loops == 0:
        if sum(curr_combination) == 100 and curr_combination not in all_combinations and all(value < 70 and value > 10 for value in curr_combination):
            all_combinations.append(curr_combination)
        return
    for i in range(0, 100 + window, window):
        generate_combinations(curr_combination + [i], remaining_loops - 1)

generate_combinations([], n)
print(all_combinations)

[[15, 20, 65], [15, 25, 60], [15, 30, 55], [15, 35, 50], [15, 40, 45], [15, 45, 40], [15, 50, 35], [15, 55, 30], [15, 60, 25], [15, 65, 20], [20, 15, 65], [20, 20, 60], [20, 25, 55], [20, 30, 50], [20, 35, 45], [20, 40, 40], [20, 45, 35], [20, 50, 30], [20, 55, 25], [20, 60, 20], [20, 65, 15], [25, 15, 60], [25, 20, 55], [25, 25, 50], [25, 30, 45], [25, 35, 40], [25, 40, 35], [25, 45, 30], [25, 50, 25], [25, 55, 20], [25, 60, 15], [30, 15, 55], [30, 20, 50], [30, 25, 45], [30, 30, 40], [30, 35, 35], [30, 40, 30], [30, 45, 25], [30, 50, 20], [30, 55, 15], [35, 15, 50], [35, 20, 45], [35, 25, 40], [35, 30, 35], [35, 35, 30], [35, 40, 25], [35, 45, 20], [35, 50, 15], [40, 15, 45], [40, 20, 40], [40, 25, 35], [40, 30, 30], [40, 35, 25], [40, 40, 20], [40, 45, 15], [45, 15, 40], [45, 20, 35], [45, 25, 30], [45, 30, 25], [45, 35, 20], [45, 40, 15], [50, 15, 35], [50, 20, 30], [50, 25, 25], [50, 30, 20], [50, 35, 15], [55, 15, 30], [55, 20, 25], [55, 25, 20], [55, 30, 15], [60, 15, 25], [60, 

In [47]:
model.eval()

Seq2Seq(
  (encoder): LSTM(10, 32)
  (decoder): LSTM(10, 32)
  (fc): Linear(in_features=32, out_features=10, bias=True)
)

In [51]:
rand_input_seq, rand_output_seq, actual_alloy, actual_alloy_d_max = random_training_example(X_train, y_train)

In [52]:
output_seq = model(rand_input_seq, rand_input_seq)
generated_alloy = combine_elements_and_composition_to_alloy(rand_input_seq[0].tolist(), list(np.array(output_seq[0].tolist()) * 100))
generated_alloy_d_max = predict_d_max(generated_alloy)

In [53]:
print(generated_alloy, generated_alloy_d_max)

Al30.6Ce48.6Co20.6 3.0


In [54]:
all_elements = ["Al", "Ce", "Co"]
max_predicted_d_max = float('-inf')
max_alloy = ""
for composition in all_combinations:
    iterative_alloy = combine_elements_and_composition_to_alloy_already_string(all_elements, composition)
    d_max_predicted = predict_d_max(iterative_alloy)
    if d_max_predicted > max_predicted_d_max:
        max_predicted_d_max = d_max_predicted
        max_alloy = iterative_alloy
print(iterative_alloy, max_predicted_d_max)

Al65Ce20Co15 4.0
